In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import glob
import keras
from functools import reduce

Using TensorFlow backend.
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//miniconda3/envs/PTSD/lib/python3.7/site-packages/tenso

In [2]:
path = r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Windowed Data' # use your path
all_files = glob.glob(path + "/*.csv")

In [5]:
li=[]
frame=[]

In [8]:
for filename in all_files:
    df1 = pd.read_csv(filename, index_col=None, header=0)
    stress=df1.groupby('windowno', as_index=False)['ptsd_moment'].max()
    acc_x=df1.groupby('windowno', as_index=False)['acc_x'].mean()
    acc_y=df1.groupby('windowno', as_index=False)['acc_y'].mean()
    acc_z=df1.groupby('windowno', as_index=False)['acc_z'].mean()
    
    acc_xmin=df1.groupby('windowno', as_index=False)['acc_x'].min()
    acc_xmin=acc_xmin.rename(columns={"acc_x": "acc_xmin"})
    acc_ymin=df1.groupby('windowno', as_index=False)['acc_y'].min()
    acc_ymin=acc_ymin.rename(columns={"acc_y": "acc_ymin"})
    acc_zmin=df1.groupby('windowno', as_index=False)['acc_z'].min()
    acc_zmin=acc_zmin.rename(columns={"acc_z": "acc_zmin"})
    
    acc_xmax=df1.groupby('windowno', as_index=False)['acc_x'].max()
    acc_xmax=acc_xmax.rename(columns={"acc_x": "acc_xmax"})
    acc_ymax=df1.groupby('windowno', as_index=False)['acc_y'].max()
    acc_ymax=acc_ymax.rename(columns={"acc_y": "acc_ymax"})
    acc_zmax=df1.groupby('windowno', as_index=False)['acc_z'].max()
    acc_zmax=acc_zmax.rename(columns={"acc_z": "acc_zmax"})
    
    acc_xsd=df1.groupby('windowno').agg(np.std,ddof=0)['acc_x']
    acc_xsd=pd.DataFrame(acc_xsd)
    acc_xsd=acc_xsd.rename(columns={"acc_x": "acc_xsd"})

    acc_ysd=df1.groupby('windowno').agg(np.std,ddof=0)['acc_y']
    acc_ysd=pd.DataFrame(acc_ysd)
    acc_ysd=acc_ysd.rename(columns={"acc_y": "acc_ysd"})
    
    acc_zsd=df1.groupby('windowno').agg(np.std,ddof=0)['acc_z']
    acc_zsd=pd.DataFrame(acc_zsd)
    acc_zsd=acc_zsd.rename(columns={"acc_z": "acc_zsd"})
    
    
    participant = df1['participant'].iloc[0]
    data=reduce(lambda x,y: pd.merge(x,y, on='windowno', how='outer'),
       [acc_x,acc_y,acc_z,acc_xmin,acc_ymin,acc_zmin,acc_xmax,acc_ymax,acc_zmax,acc_xsd
        ,acc_ysd,acc_zsd,stress])
    data['participant']= participant
    li.append(data)
frame = pd.concat(li, axis=0, ignore_index=True)

In [10]:
frame['acc_xrange']= frame['acc_xmax']-frame['acc_xmin']
frame['acc_yrange']= frame['acc_ymax']-frame['acc_ymin']
frame['acc_zrange']= frame['acc_zmax']-frame['acc_zmin']

In [11]:
frame

,windowno,acc_x,acc_y,acc_z,acc_xmin,acc_ymin,acc_zmin,acc_xmax,acc_ymax,acc_zmax,acc_xsd,acc_ysd,acc_zsd,ptsd_moment,participant,acc_xrange,acc_yrange,acc_zrange
0,2914,2.955482,4.648582,7.622101,0.761356,0.529119,6.919243,5.707777,5.640739,9.809045,1.861203,1.086771,0.560615,1.0,13,4.946421,5.111621,2.889802
1,2915,2.016756,3.831122,7.312012,0.339977,0.916979,0.636858,7.386112,10.220847,9.885658,2.080554,3.371106,3.236376,1.0,13,7.046136,9.303867,9.248801
2,2917,6.807386,4.943069,2.055135,0.572214,0.684742,0.495600,11.070788,10.273519,6.069301,3.759313,3.459381,1.228866,1.0,13,10.498574,9.588777,5.573701
3,2918,3.196858,4.363433,5.889178,0.251391,0.588974,0.509965,10.192116,10.330981,9.856929,3.415119,3.281564,3.509966,1.0,13,9.940725,9.742007,9.346964
4,2922,4.528992,7.062596,3.297055,0.351948,2.154782,0.519542,9.770737,10.041282,7.189788,3.292521,2.005847,2.147173,0.0,13,9.418790,7.886500,6.670246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13339,10624,9.649478,0.667888,2.932193,9.643845,0.656011,2.925715,9.653421,0.675165,2.940080,0.002215,0.004644,0.002972,0.0,3,0.009576,0.019154,0.014365
13340,10625,9.651067,0.676242,2.931860,9.643845,0.670377,2.925715,9.653421,0.679953,2.935291,0.002491,0.003073,0.002202,0.0,3,0.009576,0.009577,0.009577
13341,11159,3.770620,7.109995,4.264486,0.165200,1.877054,0.375890,8.786720,10.477027,8.916007,3.077231,2.257510,2.534541,0.0,3,8.621520,8.599973,8.540117
13342,11413,5.701323,5.477049,4.062128,0.124498,2.159570,0.732626,8.559271,8.956709,9.284714,2.655217,1.988869,2.832968,0.0,3,8.434773,6.797139,8.552088


In [12]:
frame.to_csv (r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Angular Accerelation Features/Angular Accerelation Features_dataframe.csv', 
              index = None, header=True) #Don't forget to add '.csv' at the end of the path